# 6. Cleaning Real Data

In [1]:
import pandas as pd

---

**Questions**
* How do you clean an example dataset?
* How do you deal with missing data?
* How do you fix column type mismatches?

**Objectives**
* Clean an example dataset using both previously described concepts and some new ones.

# Cleaning Real Data

The previous lessons have focused on key concepts with toy datasets. The remaining lessons and examples will be focused on an actual dataset from the Hawaiian Ocean Time-Series (HOT) data website ([Link to HOT Data Website](https://hahana.soest.hawaii.edu/hot/hot-dogs/)). 

The Hawaiian Ocean Time-Series has been collecting samples from station ALOHA located just North of Oahu since 1988. The map below shows the exact location where the samples we will be using originate from.

![HOT Location](https://www.soest.hawaii.edu/HOT_WOCE/img/map1image-rev2.png)

(Original image from: <https://www.soest.hawaii.edu/HOT_WOCE/bath_HOT_Hawaii.html>)


Here, we will be using data from HOT between the 1st of January 2010 to the 1st of January 2020. This  data comes from bottle extractions between depths 0 to 500m. The environmental variables that we will be looking at include:

| Column name     | Environmental Variable It Represents   |
| --------------- | -------------------------------------- |
| botid #         | Bottle ID                              |
| date mmddyy     | Date                                   |
| press dbar      | Pressure                               |
| temp ITS-90     | Temperature                            |
| csal PSS-78     | Salinity                               |
| coxy umol/kg    | Oxygen concentration                   |
| ph              | pH                                     |
| phos umol/kg    | Phosphate concentration                |
| nit umol/kg     | Nitrate + Nitrite concentration        |
| no2 nmol/kg     | Nitrite concentration                  |
| doc umol/kg     | Dissolved Organic Carbon concentration |
| hbact # 1e5/ml | Heterotrophic Bacteria concentration   |
| pbact # 1e5/ml | Prochlorococcus numbers                |
| sbact # 1e5/ml | Synechococcus numbers                  |

The data contains over 20000 individual samples. Before we analyze the data we are going to clean it up. Then, in the next lesson we will analyze and visualize it. To do this we will be using concepts we have already learned while also introducing some new concepts. 

## `DataFrame` Content Cleanup

During our initial clean up we will only load in the first few rows of our dataset entire `DataFrame`. This will make it easier to work with and less daunting.

In [2]:
pd.read_csv("./data/hot_dogs_data.csv", nrows=5)

,botid #,date mmddyy,press dbar,temp ITS-90,csal PSS-78,coxy umol/kg,ph,phos umol/kg,nit umol/kg,doc umol/kg,hbact #*1e5/ml,pbact #*1e5/ml,sbact #*1e5/ml,no2 nmol/kg,
0,2190200124,30910,5.5,23.0629,35.2514,214.1,-9,0.10,0.03,-9,-9,-9,-9,-9,NaN
1,2190200123,30910,59.6,23.0670,35.2506,214.6,-9,0.11,0.06,-9,-9,-9,-9,-9,NaN
2,2190200122,30910,90.7,21.7697,35.1897,213.4,-9,0.12,0.08,-9,-9,-9,-9,-9,NaN
3,2190200121,30910,119.4,20.7957,35.1666,208.5,-9,0.15,0.50,-9,-9,-9,-9,-9,NaN
4,2190200120,30910,153.6,20.1517,35.2192,204.6,-9,0.15,1.15,-9,-9,-9,-9,-9,NaN


From this we can see a few things:

- There are a lot of -9 values
- These actually denote Null values in the dataset
- The last column (to the right of the no2 column) doesn't seem to contain a header or any data

Both of these issues can easily be fixed using Pandas based on what we've learn previously.

To start off let's fix the first problem we saw which was was the large number of -9 values in the dataset. These are especially strange for some of the columns e.g. how can there be a negative concentration of hbact i.e. heterotrophic bacteria? This is a stand in for places where no measurement was obtained.

## 1 - Exercise: Treating -9 values as NaN values when loading data

Let's put it all together now.  To start off try fixing the read_csv() method so that all -9 values are treated as NaN values. The temperature column could technically contain -9 values. However, all temperature measurements were above 0 so this is not an issue. 


In [ ]:
# Change the code so that when we load the data any -9 value is treated as a NaN
pd.read_csv("./data/hot_dogs_data.csv", nrows=5,  <ADD YOU ANSWER HERE>)

With this we have fixed the problematic -9 values from our initial `DataFrame`.

### Column with All Missing Values


The second problem we identified was that there was an extra column (with no header) made up of only `NaN` values. This is probably an issue with the original file and if we were to take a look at the raw .csv file we find that each row ends with a ','. This causes `read_csv` to assume that there is another column with no data since it looks for a new line character (`\n`) to denote when to start a new row.

There are various methods to deal with this. However, we are going to use a relative simple method that we've already learn. As we discussed in a previous lesson pandas `DataFrame`s have a method to drop columns (or indexes) called `drop`. Where if we provide it with the correct arguments it can drop a column based on its name. Knowing this we can chain our `read_csv` method with the drop method so that we load in the blank column and then immediately remove it.

## 2 - Exercise: Dropping the blank column

The final command we will be using can be seen below. However, the columns parameter is missing any entries in its list of columns to drop. You will be fixing this by adding the name of the column that is empty (hint: the name isn't actually empty).

~~~python
pd.read_csv("./data/hot_dogs_data.csv", nrows=5, na_values=-9).drop(columns=[])
~~~

To get the name of the column we will want to utilize a `DataFrame` attribute that we have already discussed that provides us with the list of names in the same order they occur in the `DataFrame`.

In [ ]:
#  Read the data into a variable called df and print the column names 
# to find the actual name of the colunn with all missing values
pd.read_csv("./data/hot_dogs_data.csv", nrows=5)

In [ ]:
# Change the code below so that you drop the blank column
pd.read_csv("./data/hot_dogs_data.csv", nrows=5).drop(columns=[<ADD NAME OF COLUMN TO REMOVE HERE>])

In [ ]:
# Incorporate all the changes you have made above into a single read_csv line and save the resulting dataframe
# to the df variable
df = pd.read_csv("./data/hot_dogs_data.csv")
df

With this we have fixed some of the initial issues related to our dataset.


### Adding a Row Index

One final thing that we are going to do that is not quite "clean up" but nonetheless important is to set our index column when we load the data. We will use the 'botid #' column as an index. 


## 3 - Exercise: Setting the index column when loading data

To set the index column we can use a parameter in `read_csv` that was mentioned in a previous lesson. See if you can remember it!

In [ ]:
# Change the following code to take your already filtered DataFrame 
# from above and set the index column to botid #

df = pd.read_csv("./data/hot_dogs_data.csv", nrows=5)<ADD ANSWER HERE>
df

With our initial cleanup done we can now save the current version of our `DataFrame` to the `df` variable. This `df` variable will be used for the next two sections. Make sure you remove the `nrows=5` parameter since we want to load the whole `DataFrame` starting in the next section.


In [ ]:
# 1. Read the file
# 2. Replace -9 with missing value
# 3. Remove the empty column
# 4. Set the index as the `botid #` column

df = pd.read_csv(<...>)
df

---

## `DataFrame` Column Types

Now that we have fixed the initial issues we could glean from an initial look at the data we can take a look at the types that Pandas assumed for each of our columns. To do this we can access the .dtypes attribute.

###### Python

~~~python
df.dtypes
~~~

###### Output

~~~
 date mmddyy         int64
 time hhmmss         int64
 press dbar        float64
 temp ITS-90       float64
 csal PSS-78       float64
 coxy umol/kg      float64
 ph                float64
 phos umol/kg      float64
 nit umol/kg       float64
 doc umol/kg       float64
 hbact #*1e5/ml    float64
 pbact #*1e5/ml    float64
 sbact #*1e5/ml    float64
 no2 nmol/kg       float64
dtype: object
~~~

Most of the columns have the correct type with the exception of the 'date mmddyy' column that has the int64 type. Pandas has a built in type to format date and time columns and conversion of the date column to this datetime type will help us later on.

To change the type of a column from an int64 to a datetime type is a bit more difficult than chaging from an int64 to float64. This is because we both need to tell Pandas the type that we want it to convert the column's data to and the format that it is. For our data this is MMDDYY which we can give to Pandas using `format='%m%d%y'`. This format parameter is similar to the one used in native python. More information can be found on the `to_datetime` method doc ([Link to datetime method docs](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)).

The code bit below creates a new column called 'date' that contains the same data for each row as is found in the 'date mmddyy' column but instead with the datetime64 type. It will **not** delete the original 'date mmddyy' column.

###### Python

~~~python
df["date"] = pd.to_datetime(df['date mmddyy'], format='%m%d%y')
df.dtypes
~~~

###### Output

~~~
date mmddyy                int64
time hhmmss                int64
press dbar               float64
temp ITS-90              float64
csal PSS-78              float64
coxy umol/kg             float64
ph                       float64
phos umol/kg             float64
nit umol/kg              float64
doc umol/kg              float64
hbact #*1e5/ml           float64
pbact #*1e5/ml           float64
sbact #*1e5/ml           float64
no2 nmol/kg              float64
date              datetime64[ns]
dtype: object
~~~

We can see from the output that we have all of our previous columns with the addition of a 'date' column with the type datetime64. If we take a look at the new column we can see that it has a different formatting compared to the 'date mmddyy' column

###### Python

~~~python
df["date"]
~~~

###### Output

~~~
botid #
2190200124   2010-03-09
2190200123   2010-03-09
2190200122   2010-03-09
2190200121   2010-03-09
2190200120   2010-03-09
                ...    
3170200706   2019-12-20
3170200705   2019-12-20
3170200704   2019-12-20
3170200703   2019-12-20
3170200702   2019-12-20
~~~
Name: collection_date, Length: 21222, dtype: datetime64[ns]

In [20]:
df["date"] = pd.to_datetime(df['date mmddyy'], format='%m%d%y')

Now that we've added this column (which contains the same data as found in 'date mmddyy' just in a different format) there is no need for the original date mmddyy column so we can drop it.

In [21]:
df = df.drop(columns=["date mmddyy"])

---

### Reordering the Columns

You notice the the columns in `df` are not conveiently sorted. Instead, you would like to see the columns names as:
```
[
 'botid #', 'date', 'press dbar', 'temp ITS-90', 'csal PSS-78',
 'coxy umol/kg', 'ph', 'phos umol/kg', 'nit umol/kg', 'doc umol/kg',
 'hbact #*1e5/ml', 'pbact #*1e5/ml', 'sbact #*1e5/ml', 'no2 nmol/kg',
] 
```       
Given a simple dataframe as follows



In [ ]:
example_df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
example_df

You can reorder the columns simply by indexing into the DataFrame in the desired column order. For example.


In [ ]:
new_order = ["B", "A"]
example_df[new_order]

### Exercise 4

Based on the above, can you reorder `df` so that the column order is permanently set to the following: 

```
new_order = [
 'botid #', 'date', 'press dbar', 'temp ITS-90', 'csal PSS-78',
 'coxy umol/kg', 'ph', 'phos umol/kg', 'nit umol/kg', 'doc umol/kg',
 'hbact #*1e5/ml', 'pbact #*1e5/ml', 'sbact #*1e5/ml', 'no2 nmol/kg',
] 
```

In [22]:
new_order = [
 'botid #', 'date', 'press dbar', 'temp ITS-90', 'csal PSS-78',
 'coxy umol/kg', 'ph', 'phos umol/kg', 'nit umol/kg', 'doc umol/kg',
 'hbact #*1e5/ml', 'pbact #*1e5/ml', 'sbact #*1e5/ml', 'no2 nmol/kg',
] 
<Update the column order here>

## `DataFrame` Overview

Lastly, we will get an overview of our `DataFrame` as a final way of checking to see if anything is wrong. To do this we can use the `describe()` method which we discussed earlier.

In [ ]:
# Get an overview of the data found in each column
df.describe()

# Summary

With this we've clean up our initial dataset. To summarize we have:

1. Replaced the -9 placeholder for Null values with NaN values.
2. Fixed an issue with an extra column containing no data.
3. Added a custom row index.
4. Converted the data in 'date mmddyy' to a Pandas supported datetime type.
5. Dropped 'date mmddyy' column since the new 'date' column contains a more appropriate data type.
6. Reorder the column names of a dataframe.


---

## Solutions

In [ ]:
# Exercise 1

# pd.read_csv("./data/hot_dogs_data.csv", nrows=5,  na_values="-9)

In [10]:
# Exercise 2

# pd.read_csv("./data/hot_dogs_data.csv", nrows=5).columns
# pd.read_csv("./data/hot_dogs_data.csv", nrows=5).drop(columns=[" "])
# df = pd.read_csv("./data/hot_dogs_data.csv", na_values='NaN').drop(columns=[" "])
# df

In [16]:
# Exercise 3

# df = pd.read_csv("./data/hot_dogs_data.csv", na_values='NaN').drop(columns=[" "])
# df.set_index("botid #")

In [ ]:
# Exercise 4

# df[new_order]
# df